In [3]:
import numpy as np

#Quantum computing (qiskit package)
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import transpile
from qiskit.circuit import Parameter
from qiskit.providers.aer import AerSimulator

#Machine learning (scikit-learn package)
from sklearn import svm

#root finding problem
from scipy.optimize import root_scalar


In [ ]:
def GroundState(N,J):
    #Creating the Ground State Gate

    qr_gs = QuantumRegister(N, name='q_U')
    gate_gs = QuantumCircuit(qr_gs, name='GS_gate')

    #list of parameters for the GS Gate (obtained from the VQE)
    theta = Parameter('θ')
    theta_range = np.linspace(0, 2*np.pi, 20)

    #provisory gates (just to write something)
    gate_gs.rz(theta, range(N))

    #binding parameter from the list
    gate_gs = gate_gs.bind_parameters({theta: theta_range[J]})
    gate_gs_inst = gate_gs.to_instruction()

    return gate_gs_inst

In [ ]:
def GroundState_dag(N,J):
    #Creating the conjugate transpose Ground State Gate 

    qr_gs_dag = QuantumRegister(N, name='q_U^dag')
    gate_gs_dag = QuantumCircuit(qr_gs_dag, name='GS_Gate_dag')

    #list of parameters for the GS Gate^dag (obtained from the VQE)
    theta = Parameter('θ')
    theta_range = np.linspace(0, 2*np.pi, 20)

    #provisory gates (just to write something)
    gate_gs_dag.rz(theta, range(N))

    #binding parameter from the list
    gate_gs_dag = gate_gs_dag.bind_parameters({theta: theta_range[J]})
    gate_gs_dag_inst = gate_gs_dag.to_instruction()

    return gate_gs_dag_inst

In [ ]:
def QuantumKernel(N,i,j):
    #Creating the Quantum-Kernel circuit

    #Defining the register of the Quantum-Kernel circuit
    qr = QuantumRegister(N, 'q')
    cr = ClassicalRegister(N, 'c')
    qc = QuantumCircuit(qr, cr)
    
    GSgate = GroundState(N,i)
    GSgate_dag = GroundState_dag(N,j)
    qc.append(GSgate, qr)
    qc.append(GSgate_dag, qr)

    qc.barrier()
    qc.measure_all(add_bits=False)

    return qc

In [9]:
def distance(J, clf):
    #distance of a point J from the hyperplane (boundary) defined via SVM

    y = clf.decision_function(J)
    w_norm = np.linalg.norm(clf.coef_)
    dist = y / w_norm

    return dist

In [8]:
def critical_J(K):
    #find the point J on the boundary of the classification defined by the SVM
    
    #optimize for the svm.SVC (see https://scikit-learn.org/stable/modules/svm.html)
    delta1 = np.array(delta1, dtype=np.float64, order='C')

    #SVM algorithm
    clf = svm.SVC(kernel = 'precomputed')
    clf.fit(K, delta1)
    
    #finding Jc = finding the root of the distance function
    d = root_scalar(distance, clf, x0=0.2, x1=1.7) 
        
    if not d.converged:
        raise Exception(d.flag)
    else:
        return d.root
    

In [ ]:
def plot JvsN():

In [ ]:
def plot Kmatrix():

In [ ]:
#set the backend
simulator = AerSimulator()

#inizialize the intervals of J for the training of the SVM
delta1 = [0.8 + x*0.002 for x in range(51)] + [1.2 + x*0.002 for x in range(51)]
delta2 = [0.6 + x*0.002 for x in range(51)] + [1.6 + x*0.002 for x in range(51)]

delta = delta1
nshots = 1000
Nmax = 1200
jc = {}

#N number of sites, i and j are the ground states for different J in H(J)
for N in range (10,Nmax,50):
    for i in range(len(delta)):
        for j in range(i:len(delta)):
            
            if i == j:
                K[i,i] = 1
            else:
                circuit = QuantumKernel(N,delta[i],delta[j])

                #transpile, run the job and grab the results
                compiled_circuit = transpile(circuit, simulator)
                job = simulator.run(compiled_circuit, shots=nshots)
                result = job.result()

                #I'm interested only in the frequency of the all zeros - qbit output
                counts = result.get_counts()
                K[i,j] = counts[N*'0'] / nshots
                K[j,i] = counts[N*'0'] / nshots

    K = np.sqrt(K)
    #call the SVM
    jc['N'] = critical_J(K)